<a href="https://colab.research.google.com/github/erlonL/series-temporais/blob/main/ST_Projeto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance snscrape pandas_market_calendars

In [44]:
import yfinance as yf
import snscrape.modules.twitter as sntwitter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_market_calendars as mcal
import pandas_datareader.data as web

## ETAPA 1: Coleta e Tratamento de Dados

S&P 500, abreviação de Standard & Poor's 500, ou simplesmente S&P, trata-se de um índice composto por quinhentos ativos cotados nas bolsas de NYSE ou NASDAQ, qualificados devido ao seu tamanho de mercado, sua liquidez e sua representação de grupo industrial.

In [46]:
# Defina o ticker para o índice S&P 500
ticker = '^BVSP'

# Crie um objeto Ticker
index = yf.Ticker(ticker)

# Defina o intervalo de datas
start_date = '2022-07-01'
end_date = '2023-03-30'

# Obtenha os dados históricos
data = index.history(start=start_date, end=end_date)

# deixar apenas a data (y/m/d) na coluna date
data.index = data.index.date

# Exiba os primeiros registros para verificação
print(data.head())

               Open      High      Low     Close    Volume  Dividends  \
2022-07-01  98542.0   99340.0  97231.0   98954.0  11609800        0.0   
2022-07-04  98952.0   99353.0  98264.0   98609.0   6279300        0.0   
2022-07-05  98608.0   98608.0  96499.0   98295.0  13358800        0.0   
2022-07-06  98294.0   99141.0  97423.0   98719.0  13348200        0.0   
2022-07-07  98722.0  101420.0  98722.0  100730.0  12696300        0.0   

            Stock Splits  
2022-07-01           0.0  
2022-07-04           0.0  
2022-07-05           0.0  
2022-07-06           0.0  
2022-07-07           0.0  


No artigo, foi filtrado os dados para os dias úteis do mercado. Faremos a seguir esse processamento:

In [33]:
nyse = mcal.get_calendar('NYSE')

dias_uteis = nyse.valid_days(start_date=start_date, end_date=end_date)

# Pegar so a data
dias_uteis = [dia.date() for dia in dias_uteis]

In [34]:
sp500 = data.loc[data.index.intersection(dias_uteis)]

print(sp500.head())

sp500.to_csv("sp500.csv")

                   Open         High          Low        Close      Volume  \
2022-07-01  3781.000000  3829.820068  3752.100098  3825.330078  4046950000   
2022-07-05  3792.610107  3832.189941  3742.060059  3831.389893  5076590000   
2022-07-06  3831.979980  3870.909912  3809.370117  3845.080078  4417720000   
2022-07-07  3858.850098  3910.629883  3858.850098  3902.620117  4057770000   
2022-07-08  3888.260010  3918.500000  3869.340088  3899.379883  3521620000   

            Dividends  Stock Splits  
2022-07-01        0.0           0.0  
2022-07-05        0.0           0.0  
2022-07-06        0.0           0.0  
2022-07-07        0.0           0.0  
2022-07-08        0.0           0.0  


In [35]:
sp500.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,187.000000,187.000000,187.000000,187.000000,1.870000e+02,187.0,187.0
mean,3940.256469,3971.879036,3909.539884,3942.570387,4.319367e+09,0.0,0.0
std,150.696957,146.413065,154.903164,152.234188,8.189322e+08,0.0,0.0
min,3520.370117,3608.340088,3491.580078,3577.030029,1.706460e+09,0.0,0.0
25%,3840.050049,3874.709961,3808.560059,3844.950073,3.905780e+09,0.0,0.0
50%,3956.229980,3977.020020,3926.590088,3957.250000,4.152250e+09,0.0,0.0
75%,4035.330078,4057.665039,3996.924927,4027.535034,4.569835e+09,0.0,0.0
max,4290.459961,4325.279785,4277.770020,4305.200195,9.354280e+09,0.0,0.0


In [40]:
query = "S&P500"
min_likes = 0
xlsx_name = "tweets_sp500.xlsx"

In [41]:
tweets = []

search_query = f'{query} min_retweets:0 min_faves:{min_likes} since:{start_date} until:{end_date}'

for tweet in sntwitter.TwitterSearchScraper(search_query).get_items():
    # Extract relevant information from the tweet
    tweet_data = {
        "tweet": tweet.content,
        "Date": tweet.date.strftime("%Y-%m-%d %H:%M:%S"),
    }

    tweets.append(tweet_data)

tweets

ERROR:snscrape.base:Error retrieving https://twitter.com/search?f=live&lang=en&q=S%26P500+min_retweets%3A0+min_faves%3A0+since%3A2022-07-01+until%3A2022-07-30&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=S%26P500+min_retweets%3A0+min_faves%3A0+since%3A2022-07-01+until%3A2022-07-30&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))"))
CRITICAL:snscrape.base:4 requests to https://twitter.com/search?f=live&lang=en&q=S%26P500+min_retweets%3A0+min_faves%3A0+since%3A2022-07-01+until%3A2022-07-30&src=spelling_expansion_revert_click failed, giving up.
CRITICAL:snscrape.base:Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=S%26P500+min_r

ScraperException: 4 requests to https://twitter.com/search?f=live&lang=en&q=S%26P500+min_retweets%3A0+min_faves%3A0+since%3A2022-07-01+until%3A2022-07-30&src=spelling_expansion_revert_click failed, giving up.